In [27]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
df=sns.load_dataset("tips")

In [3]:
df.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
X=df.drop("total_bill",axis=1)
y=df.total_bill

In [5]:
X

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
3,3.31,Male,No,Sun,Dinner,2
4,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,5.92,Male,No,Sat,Dinner,3
240,2.00,Female,Yes,Sat,Dinner,2
241,2.00,Male,Yes,Sat,Dinner,2
242,1.75,Male,No,Sat,Dinner,2


In [6]:
y

0      16.99
1      10.34
2      21.01
3      23.68
4      24.59
       ...  
239    29.03
240    27.18
241    22.67
242    17.82
243    18.78
Name: total_bill, Length: 244, dtype: float64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
cat_cols = ["sex", "smoker", "day","time"]
num_cols = ["tip", "size"]

In [12]:
num_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = "median")),
                                ('scaling', StandardScaler())])
cat_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = "most_frequent")),
                                ('encoding', OneHotEncoder())])

In [13]:
preprocessor = ColumnTransformer([("num_pipeline", num_pipeline, num_cols),
                  ("cat_pipeline", cat_pipeline, cat_cols)])

In [14]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [15]:
from sklearn.ensemble import RandomForestRegressor


In [16]:
rf=RandomForestRegressor()

In [17]:
from sklearn.model_selection import RandomizedSearchCV
params={"max_depth":[1,2,3,5,10,None],
        "n_estimators":[50,100,200,300],
          "criterion":["squared_error"]}

In [18]:
rcv=RandomizedSearchCV(rf,param_distributions=params,verbose=3,cv=5,scoring="r2")

In [19]:
rcv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=squared_error, max_depth=2, n_estimators=300;, score=0.266 total time=   3.5s
[CV 2/5] END criterion=squared_error, max_depth=2, n_estimators=300;, score=0.410 total time=   1.7s
[CV 3/5] END criterion=squared_error, max_depth=2, n_estimators=300;, score=0.477 total time=   1.7s
[CV 4/5] END criterion=squared_error, max_depth=2, n_estimators=300;, score=0.629 total time=   1.9s
[CV 5/5] END criterion=squared_error, max_depth=2, n_estimators=300;, score=0.516 total time=   1.7s
[CV 1/5] END criterion=squared_error, max_depth=None, n_estimators=100;, score=0.314 total time=   0.6s
[CV 2/5] END criterion=squared_error, max_depth=None, n_estimators=100;, score=0.497 total time=   0.3s
[CV 3/5] END criterion=squared_error, max_depth=None, n_estimators=100;, score=0.308 total time=   0.3s
[CV 4/5] END criterion=squared_error, max_depth=None, n_estimators=100;, score=0.589 total time=   1.1s
[CV 5/5] END crite

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'criterion': ['squared_error'],
                                        'max_depth': [1, 2, 3, 5, 10, None],
                                        'n_estimators': [50, 100, 200, 300]},
                   scoring='r2', verbose=3)

In [20]:
rcv.best_params_

{'n_estimators': 100, 'max_depth': 3, 'criterion': 'squared_error'}

In [21]:
rcv.best_score_

np.float64(0.46206577307652336)

In [24]:
y_pred=rcv.predict(X_test)

In [25]:
from sklearn.metrics import r2_score

In [26]:
r2_score(y_test,y_pred)

0.596746565278884